In [1]:
import torch
from torch import utils
from torchvision import datasets, transforms

import numpy as np

In [2]:
path='./MNIST_data'

#### Загружаем датасет MNIST, преобразуя в тензоры

In [3]:
train_data = datasets.MNIST(path, train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(path, train=False, download=True, transform=transforms.ToTensor())

#### Задание: преобразовать train_data таким образом, чтобы полученный датасет обладал статистиками mean = 0 и std = 1

#### Для решения задачи посчитаем текущие mean и std датасета

In [4]:
train_mean = np.mean([train_data[i][0].mean() for i in range(len(train_data))])

In [5]:
train_mean

0.13066047

In [6]:
def get_std(train_data, train_mean):
    # std = sqrt((x-mean)^2/(n-1))
    all_squared_sums = 0
    for i in range(len(train_data)):
        tensor_of_squared_sums = (train_data[i][0].flatten() - train_mean)**2
        all_squared_sums += tensor_of_squared_sums.sum()
    number_of_images = len(train_data)
    numbers_in_image = train_data[0][0].flatten().shape[0]
    std = np.sqrt(all_squared_sums/(number_of_images*numbers_in_image-1))
    return std.item()

In [7]:
train_std = get_std(train_data, train_mean)

In [8]:
train_std

0.3081079125404358

#### Добавим в transformation pipeline новый трансформ и применим к данным:

In [9]:
mnist_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((train_mean,), (train_std,)),
           ])

In [10]:
train_data_n = datasets.MNIST(path, train=True, download=True, transform=mnist_transform)
test_data_n = datasets.MNIST(path, train=False, download=True, transform=mnist_transform)

#### Посчитаем mean и std для полученных данных:

In [11]:
train_mean_n = np.mean([train_data_n[i][0].mean() for i in range(len(train_data_n))])
print(train_mean_n)

3.9513907e-08


In [12]:
train_std_n = get_std(train_data_n, train_mean_n)
print(train_std_n)

0.9999986886978149


#### Получили требуемый результат
#### "Обучались", вычисляя статистики, мы на датасете train. Посмотрим, какие статистики получились после применения преобразования к датасету test:

In [13]:
test_mean_n = np.mean([test_data_n[i][0].mean() for i in range(len(test_data_n))])
print(test_mean_n)

0.0060178316


In [14]:
test_std_n = get_std(test_data_n, test_mean_n)
print(test_std_n)

1.0076992511749268


#### Видим, что статистики очень близки, что позволяет нам сделать предположение, что датасет train и test взяты из одного распределения. 